In [1]:
from utils import Logger,adjust_learning_rate,CreateLogger,create_cosine_learning_schedule,encode_test_label,set_seed
from model import Resnet_with_uncertainty#不确定度
from dependency import *
from torch import optim
from torchcontrib.optim import SWA
from dataloader import generate_dataloader
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error # 均方误差

In [2]:
import torch.nn.functional as F
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score, roc_curve,auc,average_precision_score,precision_recall_curve
import cv2
from dependency import *
import pandas as pd
from utils import encode_test_label,Logger,encode_meta_choosed_label,encode_meta_label
from tqdm import tqdm_notebook  

test_index_df = pd.read_csv(test_index_path)
train_index_df = pd.read_csv(train_index_path)
val_index_df = pd.read_csv(val_index_path)

train_index_list = list(train_index_df['indexes'])
val_index_list = list(val_index_df['indexes'])
test_index_list = list(test_index_df['indexes'])

train_index_list_1 = train_index_list[0:206]
train_index_list_2 = train_index_list[206:]

df = pd.read_csv(img_info_path)

In [3]:
from tensorflow.keras.utils import to_categorical
import pandas as pd
from dependency import *

In [4]:
def get_label_list(image_index_list):
    diag_label_list = []
    pn_label_list = []
    str_label_list = []
    pig_label_list = []
    rs_label_list = []
    dag_label_list = []
    bwv_label_list = []
    vs_label_list = []
    meta_list = []
    meta_list_one_hot = []
    
    img_feature = []
    img_hf_feature = []
    img_vf_feature = []
    img_vhf_feature = []

    from sklearn.decomposition import PCA

    from tqdm import tqdm_notebook,tqdm
    for index_num in tqdm(image_index_list):
    #index_num = test_index_list[100]
        img_info = df[index_num:index_num+1]
        clinic_path = img_info['clinic']
        dermoscopy_path = img_info['derm']
        source_dir = '../release_v0/release_v0/images/'
        clinic_img = cv2.imread(source_dir+clinic_path[index_num])
        dermoscopy_img = cv2.imread(source_dir+dermoscopy_path[index_num])
        '''这里对于metadata需要变更'''
        meta_vector_one_hot,meta_vector = encode_meta_choosed_label(img_info,index_num)
        
        [diagnosis_label,pigment_network_label,streaks_label,pigmentation_label,regression_structures_label,
         dots_and_globules_label,blue_whitish_veil_label, vascular_structures_label],[diagnosis_label_one_hot,pigment_network_label_one_hot,
        streaks_label_one_hot,pigmentation_label_one_hot,regression_structures_label_one_hot,
        dots_and_globules_label_one_hot,blue_whitish_veil_label_one_hot, vascular_structures_label_one_hot] = encode_test_label(img_info,index_num)

        diag_label_list.append(diagnosis_label)
        pn_label_list.append(pigment_network_label)
        str_label_list.append(streaks_label)
        pig_label_list.append(pigmentation_label)
        rs_label_list.append(regression_structures_label)
        dag_label_list.append(dots_and_globules_label)
        bwv_label_list.append(blue_whitish_veil_label)
        vs_label_list.append(vascular_structures_label)
        meta_list.append(meta_vector)
        meta_list_one_hot.append(meta_vector_one_hot)



    label_dict ={'diag':diag_label_list,
                 'pn':pn_label_list,
                 'str':str_label_list,
                 'pig':pig_label_list,
                 'rs':rs_label_list,
                 'dag':dag_label_list,
                 'bwv':bwv_label_list,
                 'vs':vs_label_list}
    
    return label_dict,meta_list,meta_list_one_hot

# MLP and regression

In [5]:
#model here！
from model import MLP_with_uncertainty,MyCNN,Resnet_with_uncertainty,ResNet50_uncertainty
import torch.nn as nn

In [6]:
def train(net, train_dataloader, model_name):
    net.set_mode("train")
    train_loss = 0
    train_dia_acc = 0
    train_sps_acc = 0
    for index,(clinic_image, derm_image, meta_data, label) in enumerate(train_dataloader):
        opt.zero_grad()
        clinic_image = clinic_image.cuda()
        derm_image = derm_image.cuda()
      
        meta_data = meta_data.float().cuda()
        #print("meta_data.size:",meta_data.size())
        #Diagnostic label
        diagnosis_label = label[0].long().cuda()
        #Seven-point Checklist labels
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        
        #print("meta_data:",meta_data,meta_data.shape)
        #uncertainty
        [   (logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic),
            (logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm),
            (logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty),
            (x_clic_cos,x_derm_cos)] = net2((clinic_image,derm_image))
        #print("logit_diagnosis:",logit_diagnosis)
        logit_diagnosis = torch.nn.Softmax(dim=1)(logit_diagnosis)
        logit_pn = torch.nn.Softmax(dim=1)(logit_pn)
        logit_str = torch.nn.Softmax(dim=1)(logit_pn)
        logit_pig = torch.nn.Softmax(dim=1)(logit_pig)
        logit_rs = torch.nn.Softmax(dim=1)(logit_rs)
        logit_dag = torch.nn.Softmax(dim=1)(logit_dag)
        logit_bwv = torch.nn.Softmax(dim=1)(logit_bwv)
        logit_vs = torch.nn.Softmax(dim=1)(logit_vs)
        
        meta_data = meta_data.squeeze()
        meta_data = torch.cat([meta_data,logit_diagnosis,logit_pn,logit_str,logit_pig,logit_rs,logit_dag,logit_bwv,logit_vs],dim=1)
        meta_data = meta_data.unsqueeze(dim=1)
        
        [(logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty)] = net(meta_data)
        
        prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis),dim = 1)
        
        prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn),dim = 1)

        prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str),dim = 1)

        prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig),dim = 1)
    
        prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs),dim = 1)

        prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag),dim = 1)
        
        prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv),dim = 1)
        
        prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs),dim = 1)
        
        prediction1 = prediction1.unsqueeze(0)
        prediction2 = prediction2.unsqueeze(0)
        prediction3 = prediction3.unsqueeze(0)
        prediction4 = prediction4.unsqueeze(0)
        prediction5 = prediction5.unsqueeze(0)
        prediction6 = prediction6.unsqueeze(0)
        prediction7 = prediction7.unsqueeze(0)
        prediction8 = prediction8.unsqueeze(0)
        prediction1 = np.array(prediction1.cpu(),dtype="int")
        prediction2 = np.array(prediction2.cpu(),dtype="int")
        prediction3 = np.array(prediction3.cpu(),dtype="int")
        prediction4 = np.array(prediction4.cpu(),dtype="int")
        prediction5 = np.array(prediction5.cpu(),dtype="int")
        prediction6 = np.array(prediction6.cpu(),dtype="int")
        prediction7 = np.array(prediction7.cpu(),dtype="int")
        prediction8 = np.array(prediction8.cpu(),dtype="int")
        
        p = np.hstack((prediction1.T,prediction2.T))
        p = np.hstack((p,prediction3.T))
        p = np.hstack((p,prediction4.T))
        p = np.hstack((p,prediction5.T))
        p = np.hstack((p,prediction6.T))
        p = np.hstack((p,prediction7.T))
        p = np.hstack((p,prediction8.T))
        p = torch.tensor(p)
        
        diagnosis_label_t = diagnosis_label.unsqueeze(0)
        pn_label_t = pn_label.unsqueeze(0)
        str_label_t = str_label.unsqueeze(0)
        pig_label_t = pig_label.unsqueeze(0)
        rs_label_t = rs_label.unsqueeze(0)
        dag_label_t = dag_label.unsqueeze(0)
        bwv_label_t = bwv_label.unsqueeze(0)
        vs_label_t = vs_label.unsqueeze(0)
        pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
        str_label_t = np.array(str_label_t.cpu(),dtype="int")
        pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
        rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
        dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
        bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
        vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
        diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
        t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
        t = np.hstack((t,str_label_t.T))
        t = np.hstack((t,pig_label_t.T))
        t = np.hstack((t,rs_label_t.T))
        t = np.hstack((t,dag_label_t.T))
        t = np.hstack((t,bwv_label_t.T))
        t = np.hstack((t,vs_label_t.T))
        t = torch.tensor(t)
        
        #print("t:",t)
        #print("p:",p)
        
        certainty_b = (t == p).long()
        #print("certainty_b:",certainty_b,certainty_b.shape)
        certainty_ones = torch.tensor(np.ones(certainty_b.size()))
        uncertainty_label = []
        for i in range(certainty_ones.size(0)):
            mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
            if mean_squared_error_tmp >= 0.6:
                mean_squared_error_tmp = 1
            else:
                mean_squared_error_tmp = 0
            uncertainty_label.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
        uncertainty_label = torch.tensor(uncertainty_label)
        
        #print("uncertainty_label:",uncertainty_label)
        #print("logit_uncertainty:",logit_uncertainty)
            
        #uncertainty
        logit_uncertainty = logit_uncertainty.float().cuda()
        uncertainty_label = uncertainty_label.float().cuda()
        
        loss = torch.div(
          net.criterion(logit_diagnosis, diagnosis_label)
        + net.criterion(logit_pn, pn_label)
        + net.criterion(logit_str, str_label)
        + net.criterion(logit_pig, pig_label)
        + net.criterion(logit_rs, rs_label)
        + net.criterion(logit_dag, dag_label)
        + net.criterion(logit_bwv, bwv_label)
        + net.criterion(logit_vs, vs_label), 8)
        
        dia_acc_all = torch.div(net.metric(logit_diagnosis,diagnosis_label),int(meta_data.size(0)))
        sps_acc_all = torch.div(  net.metric(logit_pn,pn_label)
                                + net.metric(logit_str,str_label)
                                + net.metric(logit_pig,pig_label)
                                + net.metric(logit_rs,rs_label)
                                + net.metric(logit_dag,dag_label)
                                + net.metric(logit_bwv,bwv_label)
                                + net.metric(logit_vs,vs_label),int(7 * meta_data.size(0)))
        loss.backward()
        opt.step()

        train_loss += loss.item()
        train_dia_acc += dia_acc_all.item()
        train_sps_acc += sps_acc_all.item()

    train_loss = train_loss / (index + 1) # Because the index start with the value 0f zero
    train_dia_acc = train_dia_acc / (index + 1)
    train_sps_acc = train_sps_acc / (index + 1)

    return train_loss,train_dia_acc,train_sps_acc

In [7]:
def validation(net, val_dataloader, model_name):
    net.set_mode("valid")
    val_loss = 0
    val_dia_acc = 0
    val_sps_acc = 0
    for index,(clinic_image, derm_image, meta_data, label) in enumerate(val_dataloader):
        opt.zero_grad()
        clinic_image = clinic_image.cuda()
        derm_image = derm_image.cuda()
        meta_data = meta_data.float().cuda()
        
        #Diagnostic label
        diagnosis_label = label[0].long().cuda()
        #Seven-point Checklist labels
        pn_label = label[1].long().cuda()
        str_label = label[2].long().cuda()
        pig_label = label[3].long().cuda()
        rs_label = label[4].long().cuda()
        dag_label = label[5].long().cuda()
        bwv_label = label[6].long().cuda()
        vs_label = label[7].long().cuda()
        
        with torch.no_grad():
            
            #uncertainty
            [   (logit_diagnosis_clic,logit_pn_clic,logit_str_clic,logit_pig_clic,logit_rs_clic,logit_dag_clic,logit_bwv_clic,logit_vs_clic,logit_uncertainty_clic),
            (logit_diagnosis_derm,logit_pn_derm,logit_str_derm,logit_pig_derm,logit_rs_derm,logit_dag_derm,logit_bwv_derm,logit_vs_derm,logit_uncertainty_derm),
            (logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty),
            (x_clic_cos,x_derm_cos)] = net2((clinic_image,derm_image))
            #print("logit_diagnosis:",logit_diagnosis)
            logit_diagnosis = torch.nn.Softmax(dim=1)(logit_diagnosis)
            logit_pn = torch.nn.Softmax(dim=1)(logit_pn)
            logit_str = torch.nn.Softmax(dim=1)(logit_pn)
            logit_pig = torch.nn.Softmax(dim=1)(logit_pig)
            logit_rs = torch.nn.Softmax(dim=1)(logit_rs)
            logit_dag = torch.nn.Softmax(dim=1)(logit_dag)
            logit_bwv = torch.nn.Softmax(dim=1)(logit_bwv)
            logit_vs = torch.nn.Softmax(dim=1)(logit_vs)
            
            meta_data = meta_data.squeeze()
            meta_data = torch.cat([meta_data,logit_diagnosis,logit_pn,logit_str,logit_pig,logit_rs,logit_dag,logit_bwv,logit_vs],dim=1)
            meta_data = meta_data.unsqueeze(dim=1)
            [(logit_diagnosis, logit_pn, logit_str, logit_pig, logit_rs, logit_dag, logit_bwv,logit_vs,logit_uncertainty)] = net(meta_data)
            
            prediction1 = torch.argmax(torch.nn.Softmax(dim=1)(logit_diagnosis),dim = 1)

            prediction2 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pn),dim = 1)

            prediction3 = torch.argmax(torch.nn.Softmax(dim=1)(logit_str),dim = 1)

            prediction4 = torch.argmax(torch.nn.Softmax(dim=1)(logit_pig),dim = 1)

            prediction5 = torch.argmax(torch.nn.Softmax(dim=1)(logit_rs),dim = 1)

            prediction6 = torch.argmax(torch.nn.Softmax(dim=1)(logit_dag),dim = 1)

            prediction7 = torch.argmax(torch.nn.Softmax(dim=1)(logit_bwv),dim = 1)

            prediction8 = torch.argmax(torch.nn.Softmax(dim=1)(logit_vs),dim = 1)

            prediction1 = prediction1.unsqueeze(0)
            prediction2 = prediction2.unsqueeze(0)
            prediction3 = prediction3.unsqueeze(0)
            prediction4 = prediction4.unsqueeze(0)
            prediction5 = prediction5.unsqueeze(0)
            prediction6 = prediction6.unsqueeze(0)
            prediction7 = prediction7.unsqueeze(0)
            prediction8 = prediction8.unsqueeze(0)
            prediction1 = np.array(prediction1.cpu(),dtype="int")
            prediction2 = np.array(prediction2.cpu(),dtype="int")
            prediction3 = np.array(prediction3.cpu(),dtype="int")
            prediction4 = np.array(prediction4.cpu(),dtype="int")
            prediction5 = np.array(prediction5.cpu(),dtype="int")
            prediction6 = np.array(prediction6.cpu(),dtype="int")
            prediction7 = np.array(prediction7.cpu(),dtype="int")
            prediction8 = np.array(prediction8.cpu(),dtype="int")

            p = np.hstack((prediction1.T,prediction2.T))
            p = np.hstack((p,prediction3.T))
            p = np.hstack((p,prediction4.T))
            p = np.hstack((p,prediction5.T))
            p = np.hstack((p,prediction6.T))
            p = np.hstack((p,prediction7.T))
            p = np.hstack((p,prediction8.T))
            p = torch.tensor(p)

            diagnosis_label_t = diagnosis_label.unsqueeze(0)
            pn_label_t = pn_label.unsqueeze(0)
            str_label_t = str_label.unsqueeze(0)
            pig_label_t = pig_label.unsqueeze(0)
            rs_label_t = rs_label.unsqueeze(0)
            dag_label_t = dag_label.unsqueeze(0)
            bwv_label_t = bwv_label.unsqueeze(0)
            vs_label_t = vs_label.unsqueeze(0)
            pn_label_t = np.array(pn_label_t.cpu(),dtype="int")
            str_label_t = np.array(str_label_t.cpu(),dtype="int")
            pig_label_t = np.array(pig_label_t.cpu(),dtype="int")
            rs_label_t = np.array(rs_label_t.cpu(),dtype="int")
            dag_label_t = np.array(dag_label_t.cpu(),dtype="int")
            bwv_label_t = np.array(bwv_label_t.cpu(),dtype="int")
            vs_label_t = np.array(vs_label_t.cpu(),dtype="int")
            diagnosis_label_t = np.array(diagnosis_label_t.cpu(),dtype="int")
            t = np.hstack((diagnosis_label_t.T,pn_label_t.T))
            t = np.hstack((t,str_label_t.T))
            t = np.hstack((t,pig_label_t.T))
            t = np.hstack((t,rs_label_t.T))
            t = np.hstack((t,dag_label_t.T))
            t = np.hstack((t,bwv_label_t.T))
            t = np.hstack((t,vs_label_t.T))
            t = torch.tensor(t)
            
            certainty_b = (t == p).long()
            certainty_ones = torch.tensor(np.ones(certainty_b.size()))
            uncertainty_label = []
            for i in range(certainty_ones.size(0)):
                mean_squared_error_tmp = mean_squared_error(certainty_ones[i,:],certainty_b[i,:])
                if mean_squared_error_tmp >= 0.6:
                    mean_squared_error_tmp = 1
                else:
                    mean_squared_error_tmp = 0
                uncertainty_label.append([mean_squared_error_tmp]) # bigger uncertainty, bigger MSE, means the prediction is less accurate. 
            uncertainty_label = torch.tensor(uncertainty_label)
            
            #uncertainty
            logit_uncertainty = logit_uncertainty.float().cuda()
            uncertainty_label = uncertainty_label.float().cuda()
            loss = torch.div(
              net.criterion(logit_diagnosis, diagnosis_label)
            + net.criterion(logit_pn, pn_label)
            + net.criterion(logit_str, str_label)
            + net.criterion(logit_pig, pig_label)
            + net.criterion(logit_rs, rs_label)
            + net.criterion(logit_dag, dag_label)
            + net.criterion(logit_bwv, bwv_label)
            + net.criterion(logit_vs, vs_label), 8)

            dia_acc_all = torch.div(net.metric(logit_diagnosis,diagnosis_label),int(meta_data.size(0)))
            sps_acc_all = torch.div(  net.metric(logit_pn,pn_label)
                                    + net.metric(logit_str,str_label)
                                    + net.metric(logit_pig,pig_label)
                                    + net.metric(logit_rs,rs_label)
                                    + net.metric(logit_dag,dag_label)
                                    + net.metric(logit_bwv,bwv_label)
                                    + net.metric(logit_vs,vs_label),int(7 * meta_data.size(0)))
            

        val_loss += loss.item()
        val_dia_acc += dia_acc_all.item()
        val_sps_acc += sps_acc_all.item()

    val_loss = val_loss / (index + 1) # Because the index start with the value 0f zero
    val_dia_acc = val_dia_acc / (index + 1)
    val_sps_acc = val_sps_acc / (index + 1)

    return val_loss,val_dia_acc,val_sps_acc

In [8]:
def run_train(model_name,mode,i):
    log.write('** start MLP(without uncertainty) training here! **\n')
    #best_acc = 0
    es = 0
    patience = 50
    best_mean_acc = 0 
    best_loss = 300
    
    for epoch in range(epochs):
        swa_lr = cosine_learning_schedule[epoch]
        adjust_learning_rate(opt, swa_lr)

        # train_mode
        train_loss,train_dia_acc,train_sps_acc=train(net, train_dataloader,model_name)
        log.write('Round: {}, epoch: {}, Train Loss: {:.4f}, Train Dia Acc: {:.4f}, Train SPS Acc: {:.4f}\n'.format(i, epoch, train_loss,
                                                                                                         train_dia_acc,
                                                                                                         train_sps_acc))

        # validation mode
        val_loss,val_dia_acc,val_sps_acc = validation(net, val_dataloader,model_name)
        
        val_acc = (val_dia_acc + val_sps_acc) / 2
        val_mean_acc = (val_dia_acc*1 + val_sps_acc*7)/8
        
        log.write('Round: {}, epoch: {}, Valid Loss: {:.4f}, Valid Dia Acc: {:.4f}, Valid SPS Acc: {:.4f}\n'.format(i, epoch, val_loss,
                                                                                                         val_dia_acc,
                                                                                                         val_sps_acc))

     
        if val_mean_acc > best_mean_acc:
            es = 0
            best_mean_acc = val_mean_acc
            torch.save(net.state_dict(), out_dir + '/checkpoint/{}_model.pth'.format('best_mean_acc'))
            log.write('Current Best Mean Acc is {}'.format(best_mean_acc))
        #  else:
        #      es += 1
        #      print("Counter {} of {}".format(es,patience))
          
        #      if es > patience:
        #          print("Early stopping with best_mean_acc: {:.4f}".format(best_mean_acc), "and val_mean_acc for this epoch: {:.4f}".format(val_mean_acc))
        #          break
  
  
        if epoch > (epochs - swa_epoch) and epoch % 1 == 0:
            opt.update_swa()
            log.write('SWA Epoch: {}'.format(epoch))

    torch.save(net.state_dict(), out_dir+'/swa_{}_MLP_model.pth'.format(mode))

In [ ]:
if __name__ == '__main__':
    # Hyperparameters
    mode = 'multimodal'
    tech = "without-uncertainty"
    model_name = "MLP"+"_"+tech
    shape = (229, 229)
    batch_size = 5
    num_workers = 0
    data_mode = 'Normal'
    deterministic = True
    if deterministic:
        if data_mode == 'Normal':
            random_seeds = 170
        elif data_mode == 'self_evaluated':
            random_seeds = 183
    rounds = 5
    lr = 4e-4###
    epochs = 500
    swa_epoch = 250    

    train_dataloader, val_dataloader = generate_dataloader(shape, batch_size, num_workers, data_mode,"concat")
    
    for i in range(rounds):
        if deterministic:
            set_seed(random_seeds + i)
      # create logger
        print(random_seeds+i)
        log, out_dir = CreateLogger(mode, model_name,i,data_mode)
        #network 
        net = MLP_with_uncertainty(class_list).cuda()
        #net = MyCNN(class_list).cuda()
        #net = MLP_with_uncertainty(class_list).cuda()
        #net = ResNet50_uncertainty(class_list).cuda()
        weight_file = './{}_{}_{}_weight_file/{}/checkpoint/best_mean_acc_model.pth'.format("multimodal","Resnet_cos-loss and concat-without uncertainty","Normal",i)
        print(weight_file)
        net2 = Resnet_with_uncertainty(class_list).cuda()
        net2.load_state_dict(torch.load(weight_file))
      # create optimizer
        optimizer = optim.Adam(net.parameters(), lr=lr,weight_decay=5e-5)   #weight_decay   #5e-5
        opt = SWA(optimizer)  
      # create learning schedule
        cosine_learning_schedule = create_cosine_learning_schedule(epochs, lr)
        run_train(model_name,mode,i)

170

--- [START IDENTIFIER] ----------------------------------------------------------------


	out_dir      = ./multimodal_MLP_without-uncertainty_Normal_weight_file/0/



	<additional comments> ...  

	  - multimodal MLP_without-uncertainty  

	  - simple augmentation 



./multimodal_Resnet_cos-loss and concat-without uncertainty_Normal_weight_file/0/checkpoint/best_mean_acc_model.pth
** start MLP(without uncertainty) training here! **

Round: 0, epoch: 0, Train Loss: 1.1016, Train Dia Acc: 0.3092, Train SPS Acc: 0.4207

Round: 0, epoch: 0, Valid Loss: 0.9776, Valid Dia Acc: 0.5252, Valid SPS Acc: 0.6060

Current Best Mean Acc is 0.5959349773006468
Round: 0, epoch: 1, Train Loss: 0.9361, Train Dia Acc: 0.5076, Train SPS Acc: 0.5546

Round: 0, epoch: 1, Valid Loss: 0.9069, Valid Dia Acc: 0.6341, Valid SPS Acc: 0.6609

Current Best Mean Acc is 0.6575203435268344
Round: 0, epoch: 2, Train Loss: 0.8454, Train Dia Acc: 0.5928, Train SPS Acc: 0.6329

Round: 0, epoch: 2, Valid Loss: 0.8361

Round: 0, epoch: 42, Train Loss: 0.5868, Train Dia Acc: 0.7711, Train SPS Acc: 0.7655

Round: 0, epoch: 42, Valid Loss: 0.6674, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7089

Round: 0, epoch: 43, Train Loss: 0.6318, Train Dia Acc: 0.6867, Train SPS Acc: 0.7465

Round: 0, epoch: 43, Valid Loss: 0.6838, Valid Dia Acc: 0.6293, Valid SPS Acc: 0.7103

Round: 0, epoch: 44, Train Loss: 0.6002, Train Dia Acc: 0.7157, Train SPS Acc: 0.7571

Round: 0, epoch: 44, Valid Loss: 0.6794, Valid Dia Acc: 0.6065, Valid SPS Acc: 0.7057

Round: 0, epoch: 45, Train Loss: 0.6182, Train Dia Acc: 0.6884, Train SPS Acc: 0.7471

Round: 0, epoch: 45, Valid Loss: 0.6856, Valid Dia Acc: 0.6520, Valid SPS Acc: 0.7062

Round: 0, epoch: 46, Train Loss: 0.5984, Train Dia Acc: 0.7261, Train SPS Acc: 0.7484

Round: 0, epoch: 46, Valid Loss: 0.6766, Valid Dia Acc: 0.6439, Valid SPS Acc: 0.7099

Round: 0, epoch: 47, Train Loss: 0.6230, Train Dia Acc: 0.7068, Train SPS Acc: 0.7393

Round: 0, epoch: 47, Valid Loss: 0.6882, Va

Round: 0, epoch: 88, Valid Loss: 0.6593, Valid Dia Acc: 0.6732, Valid SPS Acc: 0.7154

Round: 0, epoch: 89, Train Loss: 0.6009, Train Dia Acc: 0.7189, Train SPS Acc: 0.7526

Round: 0, epoch: 89, Valid Loss: 0.6493, Valid Dia Acc: 0.7138, Valid SPS Acc: 0.7305

Current Best Mean Acc is 0.7284553002384377
Round: 0, epoch: 90, Train Loss: 0.5909, Train Dia Acc: 0.7301, Train SPS Acc: 0.7635

Round: 0, epoch: 90, Valid Loss: 0.6372, Valid Dia Acc: 0.6797, Valid SPS Acc: 0.7315

Round: 0, epoch: 91, Train Loss: 0.5961, Train Dia Acc: 0.7213, Train SPS Acc: 0.7549

Round: 0, epoch: 91, Valid Loss: 0.6593, Valid Dia Acc: 0.6732, Valid SPS Acc: 0.7173

Round: 0, epoch: 92, Train Loss: 0.5852, Train Dia Acc: 0.7333, Train SPS Acc: 0.7566

Round: 0, epoch: 92, Valid Loss: 0.6531, Valid Dia Acc: 0.6634, Valid SPS Acc: 0.7136

Round: 0, epoch: 93, Train Loss: 0.6122, Train Dia Acc: 0.7253, Train SPS Acc: 0.7532

Round: 0, epoch: 93, Valid Loss: 0.6616, Valid Dia Acc: 0.6341, Valid SPS Acc: 0.7120


Round: 0, epoch: 135, Train Loss: 0.5843, Train Dia Acc: 0.7446, Train SPS Acc: 0.7651

Round: 0, epoch: 135, Valid Loss: 0.6562, Valid Dia Acc: 0.6976, Valid SPS Acc: 0.7129

Round: 0, epoch: 136, Train Loss: 0.5875, Train Dia Acc: 0.7157, Train SPS Acc: 0.7678

Round: 0, epoch: 136, Valid Loss: 0.6584, Valid Dia Acc: 0.6780, Valid SPS Acc: 0.7066

Round: 0, epoch: 137, Train Loss: 0.5785, Train Dia Acc: 0.7614, Train SPS Acc: 0.7602

Round: 0, epoch: 137, Valid Loss: 0.6667, Valid Dia Acc: 0.6569, Valid SPS Acc: 0.7062

Round: 0, epoch: 138, Train Loss: 0.5774, Train Dia Acc: 0.7534, Train SPS Acc: 0.7593

Round: 0, epoch: 138, Valid Loss: 0.6560, Valid Dia Acc: 0.6862, Valid SPS Acc: 0.7134

Round: 0, epoch: 139, Train Loss: 0.5866, Train Dia Acc: 0.7454, Train SPS Acc: 0.7556

Round: 0, epoch: 139, Valid Loss: 0.6682, Valid Dia Acc: 0.6130, Valid SPS Acc: 0.7089

Round: 0, epoch: 140, Train Loss: 0.5917, Train Dia Acc: 0.7430, Train SPS Acc: 0.7686

Round: 0, epoch: 140, Valid Loss

Round: 0, epoch: 182, Train Loss: 0.5801, Train Dia Acc: 0.7229, Train SPS Acc: 0.7577

Round: 0, epoch: 182, Valid Loss: 0.6473, Valid Dia Acc: 0.6764, Valid SPS Acc: 0.7168

Round: 0, epoch: 183, Train Loss: 0.5980, Train Dia Acc: 0.7671, Train SPS Acc: 0.7596

Round: 0, epoch: 183, Valid Loss: 0.6590, Valid Dia Acc: 0.6846, Valid SPS Acc: 0.7157

Round: 0, epoch: 184, Train Loss: 0.5949, Train Dia Acc: 0.7518, Train SPS Acc: 0.7524

Round: 0, epoch: 184, Valid Loss: 0.6410, Valid Dia Acc: 0.6894, Valid SPS Acc: 0.7048

Round: 0, epoch: 185, Train Loss: 0.5706, Train Dia Acc: 0.7269, Train SPS Acc: 0.7592

Round: 0, epoch: 185, Valid Loss: 0.6617, Valid Dia Acc: 0.6569, Valid SPS Acc: 0.7087

Round: 0, epoch: 186, Train Loss: 0.5760, Train Dia Acc: 0.7574, Train SPS Acc: 0.7633

Round: 0, epoch: 186, Valid Loss: 0.6616, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7268

Round: 0, epoch: 187, Train Loss: 0.5983, Train Dia Acc: 0.7333, Train SPS Acc: 0.7540

Round: 0, epoch: 187, Valid Loss

...

Round: 3, epoch: 325, Valid Loss: 0.6422, Valid Dia Acc: 0.7187, Valid SPS Acc: 0.7238

SWA Epoch: 325
Round: 3, epoch: 326, Train Loss: 0.5521, Train Dia Acc: 0.7574, Train SPS Acc: 0.7799

Round: 3, epoch: 326, Valid Loss: 0.6591, Valid Dia Acc: 0.6423, Valid SPS Acc: 0.7178

SWA Epoch: 326
Round: 3, epoch: 327, Train Loss: 0.5519, Train Dia Acc: 0.7647, Train SPS Acc: 0.7805

Round: 3, epoch: 327, Valid Loss: 0.6538, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7282

SWA Epoch: 327
Round: 3, epoch: 328, Train Loss: 0.5376, Train Dia Acc: 0.7687, Train SPS Acc: 0.7850

Round: 3, epoch: 328, Valid Loss: 0.6543, Valid Dia Acc: 0.6797, Valid SPS Acc: 0.7117

SWA Epoch: 328
Round: 3, epoch: 329, Train Loss: 0.5628, Train Dia Acc: 0.7839, Train SPS Acc: 0.7774

Round: 3, epoch: 329, Valid Loss: 0.6415, Valid Dia Acc: 0.6699, Valid SPS Acc: 0.7180

SWA Epoch: 329
Round: 3, epoch: 330, Train Loss: 0.5675, Train Dia Acc: 0.7478, Train SPS Acc: 0.7640

Round: 3, epoch: 330, Valid Loss: 0.6531, Val

Round: 3, epoch: 368, Valid Loss: 0.6450, Valid Dia Acc: 0.6358, Valid SPS Acc: 0.7305

SWA Epoch: 368
Round: 3, epoch: 369, Train Loss: 0.5479, Train Dia Acc: 0.7550, Train SPS Acc: 0.7917

Round: 3, epoch: 369, Valid Loss: 0.6515, Valid Dia Acc: 0.7041, Valid SPS Acc: 0.7141

SWA Epoch: 369
Round: 3, epoch: 370, Train Loss: 0.5597, Train Dia Acc: 0.7406, Train SPS Acc: 0.7877

Round: 3, epoch: 370, Valid Loss: 0.6468, Valid Dia Acc: 0.6943, Valid SPS Acc: 0.7250

SWA Epoch: 370
Round: 3, epoch: 371, Train Loss: 0.5434, Train Dia Acc: 0.7928, Train SPS Acc: 0.7800

Round: 3, epoch: 371, Valid Loss: 0.6487, Valid Dia Acc: 0.6650, Valid SPS Acc: 0.7292

SWA Epoch: 371
Round: 3, epoch: 372, Train Loss: 0.5494, Train Dia Acc: 0.7815, Train SPS Acc: 0.7756

Round: 3, epoch: 372, Valid Loss: 0.6447, Valid Dia Acc: 0.6715, Valid SPS Acc: 0.7194

SWA Epoch: 372
Round: 3, epoch: 373, Train Loss: 0.5818, Train Dia Acc: 0.7430, Train SPS Acc: 0.7702

Round: 3, epoch: 373, Valid Loss: 0.6729, Val

Round: 3, epoch: 411, Valid Loss: 0.6532, Valid Dia Acc: 0.6992, Valid SPS Acc: 0.7329

SWA Epoch: 411
Round: 3, epoch: 412, Train Loss: 0.5681, Train Dia Acc: 0.7157, Train SPS Acc: 0.7715

Round: 3, epoch: 412, Valid Loss: 0.6676, Valid Dia Acc: 0.6780, Valid SPS Acc: 0.7082

SWA Epoch: 412
Round: 3, epoch: 413, Train Loss: 0.5758, Train Dia Acc: 0.7574, Train SPS Acc: 0.7629

Round: 3, epoch: 413, Valid Loss: 0.6540, Valid Dia Acc: 0.6374, Valid SPS Acc: 0.7247

SWA Epoch: 413
Round: 3, epoch: 414, Train Loss: 0.5444, Train Dia Acc: 0.7671, Train SPS Acc: 0.7803

Round: 3, epoch: 414, Valid Loss: 0.6442, Valid Dia Acc: 0.6846, Valid SPS Acc: 0.7364

SWA Epoch: 414
Round: 3, epoch: 415, Train Loss: 0.5685, Train Dia Acc: 0.7325, Train SPS Acc: 0.7792

Round: 3, epoch: 415, Valid Loss: 0.6473, Valid Dia Acc: 0.6748, Valid SPS Acc: 0.7245

SWA Epoch: 415
Round: 3, epoch: 416, Train Loss: 0.5522, Train Dia Acc: 0.7494, Train SPS Acc: 0.7785

Round: 3, epoch: 416, Valid Loss: 0.6717, Val

Round: 3, epoch: 454, Valid Loss: 0.6805, Valid Dia Acc: 0.6390, Valid SPS Acc: 0.7092

SWA Epoch: 454
Round: 3, epoch: 455, Train Loss: 0.5583, Train Dia Acc: 0.7390, Train SPS Acc: 0.7751

Round: 3, epoch: 455, Valid Loss: 0.6192, Valid Dia Acc: 0.7073, Valid SPS Acc: 0.7345

SWA Epoch: 455
Round: 3, epoch: 456, Train Loss: 0.5700, Train Dia Acc: 0.7494, Train SPS Acc: 0.7647

Round: 3, epoch: 456, Valid Loss: 0.6645, Valid Dia Acc: 0.6520, Valid SPS Acc: 0.7075

SWA Epoch: 456
Round: 3, epoch: 457, Train Loss: 0.5211, Train Dia Acc: 0.7888, Train SPS Acc: 0.7837

Round: 3, epoch: 457, Valid Loss: 0.6534, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7217

SWA Epoch: 457
Round: 3, epoch: 458, Train Loss: 0.5844, Train Dia Acc: 0.7116, Train SPS Acc: 0.7738

Round: 3, epoch: 458, Valid Loss: 0.6580, Valid Dia Acc: 0.6472, Valid SPS Acc: 0.7226

SWA Epoch: 458
Round: 3, epoch: 459, Train Loss: 0.5315, Train Dia Acc: 0.7486, Train SPS Acc: 0.7873

Round: 3, epoch: 459, Valid Loss: 0.6462, Val

Round: 3, epoch: 497, Valid Loss: 0.6399, Valid Dia Acc: 0.7350, Valid SPS Acc: 0.7254

SWA Epoch: 497
Round: 3, epoch: 498, Train Loss: 0.5487, Train Dia Acc: 0.7639, Train SPS Acc: 0.7736

Round: 3, epoch: 498, Valid Loss: 0.6469, Valid Dia Acc: 0.6602, Valid SPS Acc: 0.7280

SWA Epoch: 498
Round: 3, epoch: 499, Train Loss: 0.5586, Train Dia Acc: 0.7470, Train SPS Acc: 0.7748

Round: 3, epoch: 499, Valid Loss: 0.6561, Valid Dia Acc: 0.6780, Valid SPS Acc: 0.7224

SWA Epoch: 499
174

--- [START IDENTIFIER] ----------------------------------------------------------------


	out_dir      = ./multimodal_MLP_without-uncertainty_Normal_weight_file/4/



	<additional comments> ...  

	  - multimodal MLP_without-uncertainty  

	  - simple augmentation 



./multimodal_Resnet_cos-loss and concat-without uncertainty_Normal_weight_file/4/checkpoint/best_mean_acc_model.pth
** start MLP(without uncertainty) training here! **

Round: 4, epoch: 0, Train Loss: 1.0254, Train Dia Acc: 0.3229, Train SP